In [1]:
import numpy as np
import pandas as pd
from micrograd.engine import Value
import math
from micrograd.nn import Neuron, Layer,MLP

In [2]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.shape

(891, 12)

In [4]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df['Age'].fillna(df['Age'].mean(), inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_16372\1698716155.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)


In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
# 1. Sex - Label Encoding (male=0, female=1)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# 2. Embarked - One-hot encoding
embarked_dummies = pd.get_dummies(df['Embarked'], prefix='Embarked')
df = pd.concat([df, embarked_dummies], axis=1)

# 3. Extract Cabin information (rather than encoding directly)
# First letter of cabin represents deck
df['Deck'] = df['Cabin'].str[0]
df['HasCabin'] = df['Cabin'].apply(lambda x: 0 if x == 'Unknown' else 1)

# 4. Extract title from Name
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# Group rare titles
title_mapping = {
    "Mr": "Mr",
    "Miss": "Miss",
    "Mrs": "Mrs",
    "Master": "Master",
    "Dr": "Rare",
    "Rev": "Rare",
    "Col": "Rare",
    "Major": "Rare",
    "Mlle": "Miss",
    "Countess": "Rare",
    "Ms": "Miss",
    "Lady": "Rare",
    "Jonkheer": "Rare",
    "Don": "Rare",
    "Dona": "Rare",
    "Mme": "Mrs",
    "Capt": "Rare",
    "Sir": "Rare"
}
df['Title'] = df['Title'].map(title_mapping)
# One-hot encode title
title_dummies = pd.get_dummies(df['Title'], prefix='Title')
df = pd.concat([df, title_dummies], axis=1)

# 5. Drop original categorical columns we've encoded
df.drop(['Sex', 'Embarked', 'Cabin', 'Name', 'Ticket', 'Title', 'Deck'], axis=1, inplace=True)

# 6. Normalize numeric features
numeric_features = ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch']
for feature in numeric_features:
    mean = df[feature].mean()
    std = df[feature].std()
    df[feature] = (df[feature] - mean) / std

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Hp\AppData\Local\Temp\ipykernel_16372\3848764960.py:14: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [9]:
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,HasCabin,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,1,0,0.826913,-0.592148,0.432550,-0.473408,-0.502163,False,False,True,1,False,False,True,False,False
1,2,1,-1.565228,0.638430,0.432550,-0.473408,0.786404,True,False,False,1,False,False,False,True,False
2,3,1,0.826913,-0.284503,-0.474279,-0.473408,-0.488580,False,False,True,1,False,True,False,False,False
3,4,1,-1.565228,0.407697,0.432550,-0.473408,0.420494,False,False,True,1,False,False,False,True,False
4,5,0,0.826913,0.407697,-0.474279,-0.473408,-0.486064,False,False,True,1,False,False,True,False,False


In [10]:
df.shape

(891, 16)

In [11]:
# Get your target variable
y = df['Survived'].values

# Select all features except PassengerId and Survived
X_columns = [col for col in df.columns if col not in ['PassengerId', 'Survived']]
X = df[X_columns].values

print(f"X shape: {X.shape}, features: {X_columns}")
print(f"y shape: {y.shape}")

X shape: (891, 14), features: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'HasCabin', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare']
y shape: (891,)


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=22)

In [13]:
X_train.shape

(712, 14)

In [14]:
y_train.shape

(712,)

In [15]:
input_size = X_train.shape[1] 
model = MLP(input_size, [16, 8, 1])  

# Training loop
epochs = 100
learning_rate = 0.01

for epoch in range(epochs):
    # Forward pass
    total_loss = 0
    correct = 0
    
    for i in range(len(X_train)):
        # Convert inputs to Value objects
        x = [Value(xi) for xi in X_train[i]]
        
        # Get prediction
        pred = model(x)
        
        # Apply sigmoid using micrograd operations
        # sigmoid(x) = 1/(1 + e^(-x))
        sigmoid_pred = 1.0 / (1.0 + (-pred).exp())
        
        # Binary cross-entropy loss using micrograd operations
        y_true = y_train[i]
        eps = 1e-7  # Small epsilon to avoid log(0)
        
        if y_true == 1:
            loss = -(sigmoid_pred + eps).log()
        else:
            loss = -(1.0 - sigmoid_pred + eps).log()
        
        # For accuracy calculation (using data only for metrics)
        y_pred = 1 if sigmoid_pred.data > 0.5 else 0
        correct += (y_pred == y_true)
        
        # Backward pass
        model.zero_grad()
        loss.backward()
        
        # Update weights
        for p in model.parameters():
            p.data -= learning_rate * p.grad
            
        total_loss += loss.data
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss/len(X_train)}, Accuracy: {correct/len(X_train)}")

Epoch 0, Loss: 0.6183411043924799, Accuracy: 0.7134831460674157
Epoch 10, Loss: 0.39539406801587385, Accuracy: 0.8455056179775281
Epoch 20, Loss: 0.3768031949509775, Accuracy: 0.8469101123595506
Epoch 30, Loss: 0.36404642056816194, Accuracy: 0.8469101123595506
Epoch 40, Loss: 0.3543791001326349, Accuracy: 0.8567415730337079
Epoch 50, Loss: 0.3426749000350007, Accuracy: 0.8567415730337079
Epoch 60, Loss: 0.3371348266107986, Accuracy: 0.8581460674157303
Epoch 70, Loss: 0.3362850753145764, Accuracy: 0.8567415730337079
Epoch 80, Loss: 0.3278170272585902, Accuracy: 0.8595505617977528
Epoch 90, Loss: 0.32529431317547886, Accuracy: 0.8623595505617978


In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_test))


              precision    recall  f1-score   support

           0       0.81      0.87      0.84       110
           1       0.77      0.67      0.71        69

    accuracy                           0.79       179
   macro avg       0.79      0.77      0.78       179
weighted avg       0.79      0.79      0.79       179

